In [43]:
%pip install termcolor
%pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 120.8 MB/s eta 0:00:0000:01
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/5c/e9/ee572691a3fb05555bcde41826faad29ae4bc1fb07982e7f53d54a176879/scikit_learn-1.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/14/f2/10fa23f0a6b9b2439c01579ae4a9b1849d4822e972515c8f92584bfd

In [87]:
from typing import Dict, Union, List, Optional, Literal
import re
import json
import pinecone
import boto3
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from termcolor import colored
from pinecone_text.sparse import SpladeEncoder
from pinecone_text.hybrid import hybrid_convex_scale


In [2]:
# create funciton to retriev secrete

def get_secret(secret_name: str) -> Union[str, Dict]:
    secret_name = secret_name
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )
    secret_string = get_secret_value_response['SecretString']
    try:
        return json.loads(secret_string)
    except:
        return secret_string

In [3]:
INDEX_NAME = "keyword-search-demo"
pinecone_api_key = get_secret("dev/tai_service/pinecone_db/api_key")
pinecone.init(api_key=pinecone_api_key, environment="us-east-1-aws")
try:
    client = pinecone.create_index(
        name=INDEX_NAME,
        dimension=1536,
        metric="dotproduct",
        pod_type="s1.x1",
        pods=1,
        replicas=1,
    )
except pinecone.exceptions.ApiException as e:
    print(e)

(409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Sat, 22 Jul 2023 23:46:10 GMT', 'x-envoy-upstream-service-time': '600', 'content-length': '40', 'server': 'envoy'})
HTTP response body: index keyword-search-demo already exists



In [60]:
textbook_content = """Chapter 1: Introduction to Motion
In this exciting chapter, we will embark on a journey to discover the captivating principles of motion and its profound relationship to the vast world of physics.
Section 1.1: Newton's First Law of Motion: The Law of Remarkable Inertia
Newton's first law of motion, also renowned as the law of inertia, unveils the astounding notion that an object at rest possesses an unwavering determination to stay at rest, while an object in motion displays an unwavering resolve to persist in its trajectory with a constant velocity unless an external force intervenes, disrupting its path.
Section 1.2: Applications of Newton's First Law: Unveiling the Mysteries of Motion
Prepare to be astonished as we delve into captivating examples that vividly demonstrate the profound implications of Newton's first law. Imagine a car speeding along a straight road with unwavering velocity - the screeching halt caused by sudden application of brakes will unravel the hidden secrets of inertia.
Example 1.1: The Enigmatic Car Ride:
Dive deep into the intricacies as we explore the consequences of introducing an opposing force to a moving car. Marvel at the unraveling of motion as we witness a car's velocity transform in response to the sudden application of brakes.
Section 1.3: Inertia and Mass: Unveiling the Essence of Motion
One of the most enthralling aspects of motion lies in understanding the very essence of inertia, which is the relentless resistance exhibited by objects in the face of changes to their state of motion. Prepare to be captivated as we unveil the profound relationship between an object's mass and its inherently steadfast inertia.
Section 1.4: Further Examples of Newton's First Law: Unlocking the Secrets of Motion
Enter a realm where even the simplest scenarios hold awe-inspiring secrets. Picture a serene glass of water placed on a table. Prepare to be astonished as we probe the consequences of abruptly tilting the table, and witness the mysterious dance that unfolds in accordance with Newton's first law.
Section 1.5: Understanding Equilibrium: Balancing Act of Forces
Immerse yourself in the enchanting concept of equilibrium, as we journey through a state where forces harmoniously coalesce, granting objects a state of tranquil stability, resulting in zero acceleration.
Chapter 2: Forces and Motion: Harnessing the Power of Dynamics
Embark on a thrilling expedition into the realm of forces and the dynamic interplay with motion, unraveling the astonishing principles that govern the physical world.
Section 2.1: Newton's Second Law of Motion: Unleashing the Forces Within
Newton's second law of motion lies at the core of understanding the exhilarating relationship between acceleration, net force, and mass. Brace yourself for an intellectual revelation as we explore the direct proportionality of an object's acceleration to the net force applied, and the inverse proportionality to its mass.
Section 2.2: Calculating Force and Acceleration: Cracking the Code of Dynamics
Prepare to unravel the intricately woven tapestry of forces and acceleration as we embark on a journey of calculation. Let us guide you through a mesmerizing example where a mysterious box, bearing a mass of 10 kg, encounters a resolute net force of 20 N. Brace yourself for the revelation of its resulting acceleration.
Example 2.1: Box in Motion: Decoding Acceleration:
Immerse yourself in this spellbinding example as we unleash the forces acting upon a box, ultimately determining its acceleration. Brace yourself for the unveiling of the box's mesmerizing response to the applied net force.
Section 2.3: Newton's Third Law of Motion: The Realm of Equal and Opposite Reactions
Prepare to be astounded by the enigmatic world of action and reaction. Newton's third law of motion will captivate your imagination as we explore the extraordinary phenomenon that, for every action, there exists an equal and opposite reaction.
Section 2.4: Applications of Newton's Third Law: The Reverberating Ripples of Motion
Prepare to be enthralled as we delve into captivating examples that showcase the profound implications of Newton's third law. Cast your mind to a serene lake, envision a determined individual rowing a boat, and marvel at the mesmerizing ripple of motion ignited in response to each rowing action.
Example 2.2: The Rowing Symphony: Unraveling Motion:
Embark on this captivating example as we unveil the harmonious interplay between rowing and the responsive motion of the boat. Prepare to be mesmerized as we reveal the intricately choreographed dance orchestrated by Newton's third law.
Section 2.5: Momentum and Impulse: Unleashing the Hidden Forces
Prepare to be entranced by the silent forces that shape our world. Let us guide you through the enchanting concepts of momentum and impulse, as we illuminate the profound relationship between an object's mass, velocity, and the resulting change in momentum caused by external forces acting upon it for a specific duration of time.
"""

In [89]:
SMALL_OVERLAP = 100
small_splitter = RecursiveCharacterTextSplitter(keep_separator=False, chunk_size=500, chunk_overlap=SMALL_OVERLAP)
LARGE_OVERLAP = 300
large_splitter = RecursiveCharacterTextSplitter(keep_separator=False, chunk_size=1200, chunk_overlap=LARGE_OVERLAP)
embedder = OpenAIEmbeddings(openai_api_key=get_secret("dev/tai_service/openai/api_key"))

In [112]:
textbook_doc = Document(page_content=textbook_content, metadata={"chunk_size": "small"})
textbook_docs = small_splitter.split_documents([textbook_doc])
textbook_doc = Document(page_content=textbook_content, metadata={"chunk_size": "large"})
textbook_docs.extend(large_splitter.split_documents([textbook_doc]))
for doc in textbook_docs:
    print(colored(doc.page_content, "blue"))
    print(colored("=" * 100, "red"))
assignment_embeddings = embedder.embed_documents([doc.page_content for doc in textbook_docs])
sparse_embeddings = SpladeEncoder().encode_documents([doc.page_content for doc in textbook_docs])
vectors = []
section_number_pattern = r"(?:(?<=\b)|(?<=\t))(\d+(?:\.\d+)?)\b"
chapter_pattern = r'\b(chapter)\s*(\d+)\b'
current_chapter = ""
for _id, triplet in enumerate(zip(textbook_docs, assignment_embeddings, sparse_embeddings)):
    doc, embedding, sparse_embedding = triplet
    matches = re.findall(section_number_pattern, doc.page_content)
    matches = [match for match in matches if match]
    chapter_match = re.search(chapter_pattern, doc.page_content, flags=re.IGNORECASE)
    if chapter_match:
        current_chapter = (chapter_match.group(1) + " " + chapter_match.group(2)).lower()
    doc.metadata.update({"text": doc.page_content, "sections": matches, "chapter": current_chapter})
    vectors.append({
        "id": str(_id),
        "values": embedding,
        "metadata": doc.metadata,
        "sparse_values": sparse_embedding,
    })
index = pinecone.Index(INDEX_NAME)
index.upsert(vectors=vectors)


Chapter 1: Introduction to Motion
In this exciting chapter, we will embark on a journey to discover the captivating principles of motion and its profound relationship to the vast world of physics.
Section 1.1: Newton's First Law of Motion: The Law of Remarkable Inertia
Section 1.1: Newton's First Law of Motion: The Law of Remarkable Inertia
Newton's first law of motion, also renowned as the law of inertia, unveils the astounding notion that an object at rest possesses an unwavering determination to stay at rest, while an object in motion displays an unwavering resolve to persist in its trajectory with a constant velocity unless an external force intervenes, disrupting its path.
Section 1.2: Applications of Newton's First Law: Unveiling the Mysteries of Motion
Section 1.2: Applications of Newton's First Law: Unveiling the Mysteries of Motion
Prepare to be astonished as we delve into captivating examples that vividly demonstrate the profound implications of Newton's first law. Imagine a 

{'upserted_count': 19}

In [109]:
query = "can you explain section 2.3about newtons law of motion"
embedding = embedder.embed_query(query)
sparse_vector_query = SpladeEncoder().encode_queries(query)
matches = re.findall(section_number_pattern, query)
section_numbers = [match for match in matches if match]
chapter_match = re.search(chapter_pattern, query, flags=re.IGNORECASE)
if chapter_match:
    query_chapter = (chapter_match.group(1) + " " + chapter_match.group(2)).lower()
else:
    first_number_of_sections = [int(section_number.split(".")[0]) for section_number in section_numbers]
    query_chapters = [f"chapter {number}" for number in first_number_of_sections]

In [110]:
def query_db(
    embedding: List[float],
    sparse_vector: List[float],
    section_numbers: Optional[List[str]] = None,
    chapters: Optional[List[str]] = None,
    chunk_size: Optional[Literal["small", "large"]] = None,
    alpha: float = 0.5,
) -> List[Dict]:
    hybrid_dense, hybrid_sparse = hybrid_convex_scale(embedding, sparse_vector, alpha=alpha)
    meta_data_filter = {}
    if section_numbers or chapters or chunk_size:
        and_filter = []
        if section_numbers:
            and_filter.append({"sections": {"$in": section_numbers}})
        if chapters:
            and_filter.append({"chapter": {"$in": chapters}})
        if chunk_size:
            and_filter.append({"chunk_size": chunk_size})
        meta_data_filter = {"$and": and_filter}
    results = index.query(
        hybrid_dense,
        sparse_vector=hybrid_sparse,
        top_k=4,
        include_metadata=True,
        filter=meta_data_filter,
    )
    results = results.to_dict()
    matches = results["matches"]
    return matches

print(colored("Query: {}".format(query), "blue"))
print(colored("=" * 100, "red"))
print(colored("Results with section number and chapter number", "blue"))
matches = query_db(embedding, sparse_vector_query, section_numbers=section_numbers, chapters=query_chapters, chunk_size="small", alpha=0.2)
print(colored(json.dumps(matches, indent=2), "yellow"))

print(colored("Results without section number and chapter number", "blue"))
matches = query_db(embedding, sparse_vector_query, chunk_size="large", alpha=0.8)
print(colored(json.dumps(matches, indent=2), "magenta"))

Query: can you explain section 2.3about newtons law of motion
Results with section number and chapter number
[
  {
    "id": "6",
    "score": 7.92992592,
    "values": [],
    "metadata": {
      "chapter": "chapter 2",
      "chunk_size": "small",
      "sections": [
        "1.5",
        "2"
      ],
      "text": "Section 1.5: Understanding Equilibrium: Balancing Act of Forces\nImmerse yourself in the enchanting concept of equilibrium, as we journey through a state where forces harmoniously coalesce, granting objects a state of tranquil stability, resulting in zero acceleration.\nChapter 2: Forces and Motion: Harnessing the Power of Dynamics\nEmbark on a thrilling expedition into the realm of forces and the dynamic interplay with motion, unraveling the astonishing principles that govern the physical world."
    }
  }
]
Results without section number and chapter number
[
  {
    "id": "14",
    "score": 4.41276073,
    "values": [],
    "metadata": {
      "chapter": "chapter 1",
 

In [15]:
index = pinecone.Index(INDEX_NAME)
index.delete(delete_all=True)

{}

In [16]:
# confirm by hitting enter to dlete the index
confirm = input("Are you sure you want to delete the index? (y/n)")
if confirm.lower() == "y":
    pinecone.delete_index(INDEX_NAME)
    print("Index deleted.")
else:
    print("Index not deleted.")